<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width=500></td>
<img src="https://www.zigobot.ch/images/stories/virtuemart/product/Thymio_II_5288c11c8c241.jpg" width=500>
# <center>Project of Basics of mobile robotics</center>
## <center> Broccard Brendan, Ferreira Lopes Filipe, Pillet Maxime, Schramm Arthur</center>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook will be the report of our project as part of the course "Basics of mobile robotics" given by Prof. Francesco Mondada.</p>
<hr style="clear:both">

## 1. Introduction
<p>$\;\;\;\;\;\;$ In this project, the goal was to create an environment in which we would have a starting point, a goal to reach and multiple obstacles in the way. We implented the following code on our Thymio robot in order to make it go from the starting point to the goal, following the most efficient path through the initial obstacles and using its sensors to avoid sudden obstacles on its way. A camera allows us to create a map of the environment at the beginning of the program and further on to detect the robot's position and orientation. That information is first filtered with a Kalman filter and then processed by a PD controller that we created in order to make Thymio go smoothly in direction of it's desired goal. Thanks to the Kalman filter, if the camera is suddenly obstruated, we can still estimate the robot's position and orientation. We update our values in a main "while" loop that occurs every 0.1 second.
<br>
<br>$\;\;\;\;\;\;$ This is a typical map pattern with the thymio at the start, a local obstacle on its way, the final goal in green and the obstacles in black : 
    
<img src="Images/map.jpeg" alt="Drawing" style="width: 500px;"/>

</p>

## 2. Details on each section of the project

### 2.1 Vision
<p>$\;\;\;\;\;\;$ In this section, we will explain how we implemented the vision to our project. The goal is to understand the map and get the positions of the start/stop points and the corners of our obstacles. 
<br>
<br>$\;\;\;\;\;\;$ The first step of the vision is a "while" loop at the beginning of our process, even before the Thymio is placed on the map. The loop is asked to detect at least the start and stop chips which is the condition to stop executing the loop. The chips are differientiated by their HSV color code, the start is blue and the finish green. It also checks if we have initial obstacles in our circuit and computes the positions of their corners. We consider the corners shifted in order to keep a security margin that considers Thymio's width, which results in mapping the obstacles wider than they really are to ensure their complete avoidance.
<br>
<br>$\;\;\;\;\;\;$ This first vision step gives us a map with our start, stop and enlarged corners positions : 

<img src="Images/Vision_avec_legende.png" alt="Drawing" style="width: 500px;"/>

<br>$\;\;\;\;\;\;$ The second step is to detect the position and the facing angle of Thymio after we placed it on the start chip. It is continuously computed since it is updated in our main loop. To do so, we pasted a yellow chip in the front of Thymio and a red one in the back. The robot's position is considered as being in the middle of the two chips. The position of the yellow chip, coupled with the position of the red one, allows us to have the orientation of Thymio. 
<br>
<br> $\;\;\;\;\;\;$The robot's position is given by the blue circle and its orientation is given by the green line :
<br>
<img src="Images/Thymio_orientation.png" alt="Drawing" style="width: 150px;"/> 

### 2.2 Global Navigation
<p>$\;\;\;\;\;\;$ The global navigation computes the shortest path to go from a starting point to a finish point while going through our obstacle field. We chose to use the visibility graph as the pathfinding algorithm. It consists of a web of every possible routes to go from a point A to a point B passing by the angles of obstacles. Next, it calculates the shortest euclidian route and considers it as the optimized path to take. This path is given in the form of an array of points to reach in the given order.  
<br> 
<br>$\;\;\;\;\;\;$ The visibility graph, when applied to our map, returns this path :
<img src="Images/VisGraph.png" alt="Drawing" style="width: 500px;"/>

### 2.3 Filtering
<p>$\;\;\;\;\;\;$ Since the camera's datas are uncertain (due to noise or lack of precision), we use the Kalman filter based on theses datas in order to get a good estimation of the position and orientation of the robot (which are the states of Thymio that we calculate at each time step). In order to estimate the new state, we use the last position, oriention and speed of the motors (linear and rotation). Then, Thymio's states are updated, based on the last measurements, only if the camera detects the robot. This update compares the real measurements and the estimated ones, in order to make the algorithm better (by calculating the residual covariance matrix and the Kalman gain). If the camera doesn't detect Thymio, we use the prediction the localize the robot, that allows the robot to work even though the camera is hidden. The following formula represents the state space model that we use at each time step :
<img src="Images/Kalman.png" alt="Drawing" style="width: 500px;"/>

### 2.4 Local Navigation
<p>$\;\;\;\;\;\;$For this section, most of the code was inspired by the exercise 3 of the course given by Prof. Francesco Mondada. Indeed, in the motion_control.py file, we can find two founctions that are used for local avoidance : check_obstacle and local_avoidance. The first one just returns a boolean that tells us if any of the front sensors has a value that exceeds the trigger value in which case we consider that an obstacle has been detected. If so, the second function will use a system of weights inspired from the exercise mentionned above to return a certain speed value, that will be added to the basic speed of the robot, in order to make it turn smoothly and avoid the obstacle. There will be more info in the following section on motion control concerning how this speed value, that is returned by the function local_avoidance, is applied to the motors of the robot.

### 2.5 Motion control
<p>$\;\;\;\;\;\;$ Thymio has a constant speed but we need motion control to ensure smooth turns and to correct the drifting or the wheels' error. A controller seems to be the best solution to these problematics.
<br>
<br>$\;\;\;\;\;\;$ Therefore, we chose to use a proportional-derivative (PD) controller to calculate our control speed, that will influence the wheel speeds in order to orientate the robot towards its current goal. This speed is added to our constant speed (actually, the control speed is added to one of the wheel's speed and subtracted to other one) in a way that the robot never stops to turn on itself. First, our idea was to use a simple proportional controller but we expected some trouble with the local navigation since Thymio has no lateral proximity sensor. Indeed, when the robot doesn't detect the local obstacle anymore, it might suddenly turn and hit the obstacle. Adding a derivative parameter allows us to adjust even more the smoothness of the curves. 
<br>
<br>$\;\;\;\;\;\;$ The PD controller formula is 
<br> <h1><center>${v}_{control} = {K}_{p} \cdot {e}_{angle} + {K}_{d} \cdot {\Delta}_{error}$</center></h1> 
<br> where ${e}_{angle}$ is the error between Thymio's current angle and the angle he is supposed to have to reach its destination. ${\Delta}_{error}$ stands for the error variation between the previous state and the current one. We empirically found the proportional (Kp) and derivative (Kd) gains : ${K}_{p}$ = 12 and ${K}_{d}$ = 6.
<br>
<br>$\;\;\;\;\;\;$Now that we have our control speed and our avoidance speed, defined in section 2.4, with just need to add them to our basic speed (100) that keeps the robot in movement while the final goal isn't reached. Theses speeds were calculated in a way that we could add their value to the left motor speed and substract it to the right one. The following equation shows this sum that is then applied to the motors :
<br> <h1><center>${v}_{left} = {v}_{basic} + {v}_{control} \cdot {s}_{control} + {v}_{avoidance} \cdot {s}_{avoidance}$</center></h1>
<br> <h1><center>${v}_{left} = {v}_{basic} - {v}_{control} \cdot {s}_{control} - {v}_{avoidance} \cdot {s}_{avoidance}$</center></h1>
<br>where ${s}_{control}$ and ${s}_{avoidance}$ are the scales that ensure that one speed doesn't overtakes the others and that the movement of the robot stays smooth.

## 3. Global Code Execution
<p>$\;\;\;\;\;\;$ First, here is a graph that shows the general architecture of our executable code :
<img src="Images/architecture.jpg" alt="Drawing" style="width: 500px;"/>

### 3.1 Imports
<p>At first, we will start by importing the different .py files that we created and where are defined most of the functions we will use in this program :</p>

- **init_map.py** defines the functions related to a linear Kalman filter
- **motion_control.py** defines the functions related to the motion control of the robot (PD controller, local avoidance)
- **vis_graph.py** defines the functions related to the creation of the map and the visibility graph
- **update_pos.py** defines the functions that are used to detect the position and orientation of the robot with the camera
- **kalman_filter.py** defines the functions used to apply our Kalman filter to the program

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.unlock())

import time
import cv2
from init_map import *
from motion_control import *
from vis_graph import *
from update_pos import *
from kalman_filter import *

### 3.2 Definition of getters and setters
<p>In the following section, we define the getters and setters that will allow us to communicate with the robot regarding the sensors and the motor speeds :</p>

In [1]:
@tdmclient.notebook.sync_var
def get_prox_value():
    global prox_horizontal
    return prox_horizontal

@tdmclient.notebook.sync_var
def set_motor_speed(left_speed, right_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

NameError: name 'tdmclient' is not defined

### 3.3 Initialisation of the variables and constants
<p>At first, we define several constants that will be used in the initialisation :</p>

In [ ]:
#Create constant variables
next_goal_trigger = 20
delta_t = 0.05

### 3.4 Creation of the map and the global path
<p>In this section, we will explain how the map is created with the camera at the start of the program. The following cell allows to show the shortest path computed on our map.</p>

In [ ]:
VideoCap = cv2.imread("Images/Map_init.png")

#Initialisation of the variables that will register the coordinates of the starting point and the final goal
start = []
stop = []

#Creation of the map and the global path
while(len(stop)==0 or (len(start)==0)):
    start, stop = detect_start_stop(VideoCap)

global_path = initialisation(VideoCap, start, stop)
start = global_path[0]
stop = global_path[len(global_path)-1]

### 3.5 Initialisation of the variables
<p>In this section, we will initialise the different variables that will be used during the program

In [ ]:
# Set the initial motor speeds at 0
set_motor_speed(0, 0)

# Initialisation of the sensors
prox_sensors = np.array([0, 0, 0, 0, 0, 0, 0])

# Initialisation of the Kalman filter
kalman_filter = KalmanFilter(delta_t)

# Find the initial position and orientation of the robot
position, orientation = detect_thymio(start, 0, 0, 0, kalman_filter, frame)

# Initialisation of the variables that will keep in memory the position and orientation of the last iteration
old_position = position
old_orientation = orientation

# Initialisation of the variables that will keep track of the current goal position and orientation
current_goal = 1
current_goal_pos = global_path[current_goal]

distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

# Initialisation of the variables that will be updated to set the speed of the wheels
left_speed = 0
right_speed = 0

# Initialisation of the boolean that will recall if the camera detection is possible or not at each iteration
possible_detection = True

# Initialisation of the variable that will register the number of steps yet to finish an oobstacle avoidance
avoiding_steps =  0

## 3.6 Main loop
<p>Finally, we can now start implementing the program on Thymio.</p>

In [ ]:
while(True):
    # Refreshing the camera
    ret, frame = VideoCap.read()
    current_goal_pos = global_path[current_goal]
    
    # Get the values of the sensors 
    prox_sensors = get_prox_value()
    
    # Calculate the position and orientation of the robot    
    position, orientation = detect_thymio(old_position, old_orientation, left_speed, right_speed, kalman_filter, frame)
    # Drawing the position and the oriention on the image
    cv2.circle(frame, (int(position[0]), int(position[1])), 10, (255, 0, 0), 2)
    x2 = position[0] + 50 * np.cos(orientation)
    y2 = position[1] + 50 * np.sin(orientation)
    cv2.line(frame, (int(position[0]), int(position[1])), (int(x2), int(y2)),(0, 255, 0), 2)
    for i in range (len(global_path)-1):
        cv2.line(frame, (int(global_path[i][0]), int(global_path[i][1])), (int(global_path[i+1][0]), int(global_path[i+1][1])),(0, 0, 255), 2)
    cv2.imshow('image', frame)
  
    # Update the current goal orientation and the distance from the robot
    distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

    # Find the speeds to apply to the robot
    left_speed, right_speed, avoiding_steps = motion_control(orientation, old_orientation, current_goal_orientation, prox_sensors, avoiding_steps)
    
    # Apply the desired speeds to the robot
    set_motor_speed(int(left_speed), int(right_speed))
    
    # Keep in memory the position and orientation of the robot for the next 
    old_position = position
    old_orientation = orientation
    
    # Update the current goal if necessary
    if (distance_to_goal < next_goal_trigger) :
        current_goal += 1
    
    if (current_goal == len(global_path)) :
        set_motor_speed(0, 0)
        break
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
    await client.sleep(0.05)

# Close the window when the robot has reached its goal
VideoCap.release()
cv2.destroyAllWindows()

## 4. References
- visibility graph/shortest path : https://github.com/TaipanRex/pyvisgraph/tree/master/pyvisgraph
- extended kalman filter : https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/
- kalman filter, l42 project : https://github.com/L42Project/Tutoriels/blob/master/Divers/tutoriel36/KalmanFilter.py
- kalman filter, wikipedia : https://en.wikipedia.org/wiki/Kalman_filter
- obstacle avoidance, TP 3 of the course : http://localhost:8906/notebooks/Ex3/Solutions%20Week%203%20-%20Artificial%20neural%20networks.ipynb
- openCV documentation : https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html